DSC2020
----
Data Extraction

In [1]:
%%time
'''
Data reduction
'''
import csv



def makeHeadfile(rawFileName='oneHotData.csv', newFileName='FlightDelays_heads.csv', keepingRate=10, merge=False):
    data = []
    with open(rawFileName) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count==0:
                data.append(row)
            line_count += 1
            if line_count >= keepingRate:
                data.append(row)
                line_count = 1
    print(len(data))
    with open(newFileName, mode='w', newline='') as head_file:
        head_writer = csv.writer(head_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        head_writer.writerows(data)
    
    return data[0]


keepingRate = 15
raw_col = makeHeadfile(newFileName='oneHot_sample.csv',keepingRate=keepingRate)

250845
Wall time: 1min 50s


In [ ]:
%%time
'''
data merging functions
'''
import pandas as pd
import csv


# df = Original FlightDelays data + Q3 data 
def mergeQ3(baseFileName, q3FileName):
    data = pd.read_csv(baseFileName)
    
    q3 = pd.read_csv(q3FileName)
    q3 = q3.rename(columns={"CANCELLED": "CANCELED"})
    
    result = data.append(q3, sort=False)
    result = result.drop(columns=['Unnamed: 0', 'ORIGIN_CITY', 'DEST_CITY_NAME'])
    df = result.drop(result[result['CANCELED'] == 1].index)
    
    return df


# df = Original FlightDelays data + Q3 data + Market share info data
def mergeMarketShare(data, addingFileName):
    
    market = pd.read_csv(addingFileName)
    market = market.rename(columns={"Origin":"ORIGIN", "Dest":"DEST"})
    newColName = ['ORIGIN', 'DEST','nsmiles', 'fare', 'carrier_lg', 'large_ms', 'fare_lg', 'carrier_low', 'lf_ms', 'fare_low']
    data = pd.merge(data,
                     market[newColName],
                     on=['ORIGIN', 'DEST'])
    
    
    data = data.sample(frac=1)
    return data

def mergeHubs(data, hubsFileName):
    
    #make hub lists of list
    def loadHub(hubsFileName):
        hub = []
        with open(hubsFileName) as csv_file:
            hubs = csv.reader(csv_file, delimiter=',')
            t = 0
            for row in hubs:            
                if t != 0:
                    hub.append(row[0:3])
                t = 1
        return hub
    
    #compare condition and set hub values
    def hubmap(row):
        if row['CARRIER'] == 'AA':
            if row['DEST'] in carrierdict['AA']:
                return 1
            else:
                return 0
        elif row['CARRIER'] == 'UA':
            if row['DEST'] in carrierdict['UA']:
                return 1
            else:
                return 0
        elif row['CARRIER'] == 'DL':
            if row['DEST'] in carrierdict['DL']:
                return 1
            else:
                return 0
        elif row['CARRIER'] == 'WN':
            if row['DEST'] in carrierdict['WN']:
                return 1
            else:
                return 0
        elif row['CARRIER'] == 'B6':
            if row['DEST'] in carrierdict['B6']:
                return 1
            else:
                return 0
        elif row['CARRIER'] == 'AS':
            if row['DEST'] in carrierdict['AS']:
                return 1
            else:
                return 0
        elif row['CARRIER'] == '9E':
            if row['DEST'] in carrierdict['9E']:
                return 1
            else:
                return 0
        else:
            return 0
        
    hub = loadHub(hubsFileName)
    hubs = pd.DataFrame(hub, columns=['CARRIER', 'ORIGIN', 'HUBS'])
    hubs['DEST'] = hubs['ORIGIN']
    
    Carriers = hubs['CARRIER'].unique()
    carrierdict = {}
    for c in Carriers:
        tmp = hubs.loc[hubs['CARRIER'] == c]
        list0 = tmp['DEST']
        carrierdict[c] = list0.values.tolist()      
    
    data['HUBS'] = data.apply (lambda row: hubmap(row), axis=1)
    
    
    return data




In [ ]:
baseFileName = 'FlightDelays_sample.csv'
q3FileName = 'FlightDelays_2019Q3_addfinancial.csv'
addingFileName = 'AirFares.csv'
hubsFileName = 'Hubs.csv'

data = mergeQ3(baseFileName, q3FileName)
data = mergeMarketShare(data, addingFileName)
data = mergeHubs(data, hubsFileName)
data.reset_index(drop=True)

In [ ]:
'''
save data with specific attributes to reduce data size
'''

inputX = ['YEAR', 'QUARTER', 'MONTH','DAY_OF_MONTH','DAY_OF_WEEK','CARRIER','FL_NUM','Route','ORIGIN', 'DEST','DEST_STATE',
       'CRS_DEP_TIME', 'CRS_ARR_TIME', 'CRS_ELAPSED_TIME', 'DISTANCE', 'PASSENGERS', 'EMPFTE', 'NET_INCOME',
        'OP_REVENUES', 'fare', 'carrier_lg', 'large_ms', 'fare_lg','HUBS', 'ARR_DELAY_GROUP']

data = data[inputX]
data.to_csv(r'ourAtt.csv', index = False)